In [60]:
import igraph
from langdetect import detect

In [51]:
import pandas as pd

In [63]:
def parse_gdf_channel(file_name):
    with open(file_name) as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])

def parse_gdf_video(file_name):
    with open(file_name, encoding='utf-8') as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row[:14] for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])
    
    
df_video_metadata, df_video_rel = parse_gdf_video(
    'videonet_seeds1_nodes88_2019_05_08-07_42_41.gdf')   
df_video_metadata['title language'] = df_video_metadata['label VARCHAR'].apply(detect)
df_video_metadata

,nodedef>name VARCHAR,label VARCHAR,isSeed VARCHAR,seedRank INT,publishedAt INT,channelTitle VARCHAR,channelId VARCHAR,videoCategoryLabel VARCHAR,viewCount INT,likeCount INT,dislikeCount INT,dislikeLikeRatio FLOAT,favoriteCount INT,commentCount INT,title language
0,7NiYVoqBt-8,Dumbo Official Trailer,yes,1,1542246427,Walt Disney Studios,UCuaFvcY4MhZY3U43mMt1dYQ,Film & Animation,16333333,238052,11587,0.048674239241846,0,22247,en
1,0LhQ7quaZy4,10 Animated Movies That Were Changed In Other ...,no,,1520694000,Screen Rant,UC2iUwfYi_1FCGGqhOUNx-iA,Entertainment,4191315,28108,2662,0.094706133485129,0,2439,en
2,4ZoVVyrjn4Y,شاهد الفيديو لتعرف حقيقة القطط,no,,1513484626,قناة المعرفة | Almarefa channel,UC1Fnr_avpq-NtnG_R4q4xoQ,Pets & Animals,10138925,48543,11503,0.23696516490534,0,2599,ar
3,9KZKJpOPo_E,🔴 WATCH NOW! BEST CLASSIC TOM & JERRY MOMENTS ...,no,,1550861984,WB Kids,UC9trsD1jCTXXtN3xIOIU8gg,Film & Animation,10514669,77723,15996,0.20580780463955,0,,en
4,r_qdACxgZM4,KARPUZLA YAPABİLECEĞİNİZ 15 İNANILMAZ KOLAY FİKİR,no,,1553954408,5 DAKİKADA HALLET,UCMX67L1CGxutIdUynvpQnmA,Howto & Style,5327214,15784,6257,0.39641409021794,0,462,tr
5,vCfw4gcb2lI,Justice League cast ★ Before and After 2017,no,,1512136809,toptenfamous,UC_-BKZHFHcXKIQ_C5vPV4Ug,People & Blogs,10932787,28160,8025,0.28497869318182,0,1179,en
6,5abJ0rNIuqk,Super Kids Games Live – Cars for Kids Learn Co...,no,,1557287154,Super Kids Games,UCw1rV3Z6pKN3rSPklveRw2A,Education,30926,288,178,0.61805555555556,0,0,en
7,0zy-Kp_zywU,THE SECRET LIFE OF PETS 2: All NEW Trailers (2...,no,,1548950949,FilmSelect Trailer,UCT0hbLDa-unWsnZ6Rjzkfug,Entertainment,8100932,75301,2411,0.032018167089414,0,2940,en
8,7M-uqMmJdeo,تعلم الالوان بالإنجليزية للاطفال مع اغنية فينج...,no,,1509352572,cocoz toon,UCNZm7rvDZgCZrGtEBlVFidg,Film & Animation,246414403,732608,329588,0.44988315715908,0,,ar
9,x8bo5unaMpI,Dumbo 1941 - Best Memorable Moments [HD],no,,1555608370,T Pictures,UCdXj3nJ6vpyzUFpyimgSBHQ,People & Blogs,55994,138,24,0.17391304347826,0,6,en


In [55]:
    
df_metadata, df_graph = parse_gdf_channel(
    'channelnet_seeds1_nodes18_2019_05_07-12_30_18.gdf')
df_metadata.rename(columns={
    'nodedef>name VARCHAR': 'nodeid',
    'label VARCHAR': 'label'
},
    inplace=True)
df_graph.rename(columns={
    'edgedef>node1 VARCHAR': 'nodeid_1',
    'node2 VARCHAR': 'nodeid_2',
    'directed BOOLEAN': 'directed'
},
    inplace=True)
df_graph = pd.merge(df_graph,
                    df_metadata[['label', 'nodeid']],
                    left_on='nodeid_1',
                    right_on='nodeid',
                    how='right')[['label', 'nodeid_2', 'directed'
                                  ]].rename(columns={'label': 'node1'})

df_graph = pd.merge(df_graph,
                    df_metadata[['label', 'nodeid']],
                    left_on='nodeid_2',
                    right_on='nodeid',
                    how='right')[['node1', 'label', 'directed'
                                  ]].rename(columns={'label': 'node2'})
df_metadata.to_csv('disney_channel_metadata.csv')
df_metadata

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 616: character maps to <undefined>

In [53]:
df_graph.to_csv('disney_channel_relations.csv')

In [54]:
df_graph

,node1,node2,directed
0,Walt Disney Studios,Disney Family,true
1,The Muppets,Disney Family,true
2,Walt Disney Animation Studios,Disney Family,true
3,Pixar,Disney Family,true
4,Disney Movies,Disney Family,true
5,Disney,Disney Family,true
6,Oh My Disney,Disney Family,true
7,Disney Style,Disney Family,true
8,Mickey Mouse,Disney Family,true
9,Babble,Disney Family,true


In [11]:
help(igraph)

Help on package igraph:

NAME
    igraph - IGraph library.

DESCRIPTION
    @undocumented: deprecated, _graphmethod, _add_proxy_methods, _layout_method_wrapper,
                   _3d_version_for

PACKAGE CONTENTS
    _igraph
    app (package)
    clustering
    compat
    configuration
    cut
    datatypes
    drawing (package)
    formula
    layout
    matching
    remote (package)
    statistics
    summary
    test (package)
    utils
    vendor (package)

CLASSES
    builtins.object
        ARPACKOptions
        BFSIter
        Edge
        Graph
            Graph
        Vertex
    igraph.core.EdgeSeq(builtins.object)
        EdgeSeq
    igraph.core.VertexSeq(builtins.object)
        VertexSeq
    
    class ARPACKOptions(builtins.object)
     |  Class representing the parameters of the ARPACK module.
     |  
     |  ARPACK is a Fortran implementation of the implicitly restarted
     |  Arnoldi method, an algorithm for calculating some of the
     |  eigenvalues and eigenvecto